In [1]:
import pandas as pd
import json
import jsonlines
from pandas import json_normalize

# Load and parse business data with nested json

In [2]:
dict = {}
final = []
all_counter = 0;
food_counter = 0;
with open("data/yelp_academic_dataset_business.json") as f:
    data = json.loads("[" + 
        f.read().replace("}\n{", "},\n{") + 
    "]")
    
    for values in data:
        all_counter+=1
        #print(values)
        business_id = values['business_id']
        name = values['name']
        address = values['address']
        city = values['city']
        state = values['state']
        postal_code = values['postal_code']
        stars = values['stars']
        review_count = values['review_count']
        categories = values['categories']
       

        dict={'business_id': business_id, 'name': name, 'address': address, 
             'city': city,'state': state, 'postal_code': postal_code,'stars': stars, 
             'review_count': review_count,'categories': categories}
        if (categories is not None):
            #print (categories.split(", "))
            if("Food" in categories.split(", ")):
                #print (categories.split(", "))
                #print(dict)
                food_counter+=1
                final.append(dict)

In [3]:
final[0:3]

[{'business_id': '6iYb2HFDywm3zjuRg0shjw',
  'name': 'Oskar Blues Taproom',
  'address': '921 Pearl St',
  'city': 'Boulder',
  'state': 'CO',
  'postal_code': '80302',
  'stars': 4.0,
  'review_count': 86,
  'categories': 'Gastropubs, Food, Beer Gardens, Restaurants, Bars, American (Traditional), Beer Bar, Nightlife, Breweries'},
 {'business_id': 'rYs_1pNB_RMtn5WQh55QDA',
  'name': 'Chautauqua General Store',
  'address': '100 Clematis Dr',
  'city': 'Boulder',
  'state': 'CO',
  'postal_code': '80302',
  'stars': 3.5,
  'review_count': 5,
  'categories': 'Food, Shopping, Convenience Stores, Souvenir Shops'},
 {'business_id': 'HPA_qyMEddpAEtFof02ixg',
  'name': "Mr G's Pizza & Subs",
  'address': '474 Lowell St',
  'city': 'Peabody',
  'state': 'MA',
  'postal_code': '01960',
  'stars': 4.0,
  'review_count': 39,
  'categories': 'Food, Pizza, Restaurants'}]

In [4]:
food_counter, all_counter, food_counter/all_counter

(29469, 160585, 0.18351029050035805)

In [5]:
items = pd.DataFrame.from_records(final)

In [6]:
items.city.value_counts() # Portland has the most number of business

Portland           3935
Austin             3463
Vancouver          3110
Atlanta            2005
Orlando            1877
                   ... 
Wellesley Hills       1
Hanscom AFB           1
Heathrow              1
Creedmoor             1
Lower Mills           1
Name: city, Length: 395, dtype: int64

### Only focus on the  3935 business in Portland:

In [7]:
biz = items[items.city == "Portland"]
portland_biz_ids = biz.business_id.to_list() 

In [8]:
biz_json = biz.to_json(orient = "records")
biz_parsed = json.loads(biz_json)
biz_parsed[0:3]

[{'business_id': 'Npm0cjoyWwyV13OULL9qOA',
  'name': "Nana's Ice Cream Scoop Shop",
  'address': '5224 NE Garfield Ave',
  'city': 'Portland',
  'state': 'OR',
  'postal_code': '97211',
  'stars': 4.0,
  'review_count': 5,
  'categories': 'Food, Ice Cream & Frozen Yogurt'},
 {'business_id': 'ZA3u0Nu5V6TqkcYh8U0zdg',
  'name': 'B Street Coffee House',
  'address': '2190 W Burnside St',
  'city': 'Portland',
  'state': 'OR',
  'postal_code': '97210',
  'stars': 4.5,
  'review_count': 131,
  'categories': 'Restaurants, Food, Sandwiches, Breakfast & Brunch, Gluten-Free, Coffee & Tea, Bagels'},
 {'business_id': 'CBkF_Ub_nxOhOEN3Zbo_RQ',
  'name': 'Plaid Pantry',
  'address': '2010 N Killingsworth St',
  'city': 'Portland',
  'state': 'OR',
  'postal_code': '97217',
  'stars': 4.0,
  'review_count': 10,
  'categories': 'Convenience Stores, Food'}]

### Save business data to json file

In [9]:
# with open('data/yelp_academic_dataset_business_food.json', 'w') as f:
#     for obj in biz_parsed:
#         json.dump(obj, f,  separators=(", ", ": "))
#         f.write('\n')

# Load review data

In [10]:
reviews = pd.read_json("data/yelp_academic_dataset_review.json", lines=True)
reviews.head(3)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,lWC-xP3rd6obsecCYsGZRg,ak0TdVmGKo4pwqdJSTLwWw,buF9druCkbuXLX526sGELQ,4,3,1,1,Apparently Prides Osteria had a rough summer a...,2014-10-11 03:34:02
1,8bFej1QE5LXp4O05qjGqXA,YoVfDbnISlW0f7abNQACIg,RA4V8pr014UyUbDvI-LW2A,4,1,0,0,This store is pretty good. Not as great as Wal...,2015-07-03 20:38:25
2,NDhkzczKjLshODbqDoNLSg,eC5evKn1TWDyHCyQAwguUw,_sS2LBIGNT5NQb6PD1Vtjw,5,0,0,0,I called WVM on the recommendation of a couple...,2013-05-28 20:38:06


In [11]:
portland_food_reviews = reviews[reviews.business_id.isin(portland_biz_ids)]
portland_food_reviews.head(3)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
42,lJ7rzbvT-l8KO8lHfEsXsg,LV1ME-ibA2h0IGyFUUWhaQ,H_RM2u1WWGU1HkKZrYq2Ow,5,0,0,0,Incredible donuts. Sometimes you have to go ea...,2017-08-07 19:34:13
45,m-9DK7NwYedIPj1RQ_sXdw,JuM-lH05m6Ln8OPUTg8p0g,H_RM2u1WWGU1HkKZrYq2Ow,5,0,0,0,"Dont bother going to voodoo, just come here in...",2016-09-18 17:06:01
56,oL2y3mXe_qab349DPW6uZg,4cDqW9sWnZ57tsIEiElJWw,0nMbLN0B5I-2tNnQLSo86w,5,4,4,4,"""Even the mad Captain Ahab is touched by Starb...",2017-12-02 19:41:59


In [12]:
portland_food_reviews.business_id.nunique()

3935

In [13]:
portland_food_reviews.stars.value_counts()

5    196108
4     96340
3     41153
1     28104
2     27397
Name: stars, dtype: int64

In [14]:
portland_food_reviews.date.min(), portland_food_reviews.date.max()

(Timestamp('2004-12-10 01:16:29'), Timestamp('2021-01-28 09:08:12'))

In [15]:
portland_food_reviews[portland_food_reviews.date > pd.to_datetime('2011-01-01 00:00:00')].stars.value_counts() 

5    182928
4     83599
3     36219
1     26630
2     24806
Name: stars, dtype: int64

In [16]:
portland_food_reviews.business_id.value_counts()

4CxF8c3MB7VAdY8zFb2cZQ    9295
OQ2oHkcWA8KNC1Lsvj1SBA    7394
OPfgKOm_n-ajUo3qjSEgRg    4332
H_RM2u1WWGU1HkKZrYq2Ow    4055
o_L9Ss4boqq6ZEF9xeSH6Q    3721
                          ... 
x03b65sGbnxfLz4JV2w1lw       5
dcyU9_yA5TWOry5px_6OjA       5
UcvTXXBO2CKA5auE8Q-QxA       5
IYfq-zB7FHjc6UmquJ-UPQ       5
KM_PMnU1fNP7iXorysT6yg       5
Name: business_id, Length: 3935, dtype: int64

### Filter reviews in Portland since 2018 (last 3 years reviews) 
~112,381 reviews related to these 3935 restaurant business

In [17]:
portland_food_reviews_last_3_years = portland_food_reviews[portland_food_reviews.date > pd.to_datetime('2018-01-01 00:00:00')]
portland_food_reviews_last_3_years.head(3)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
71,drj-5euhU_5uVFcZ9nvsjw,onoPTc_dcSlQOOk6PUxpaw,g4CP3kgH1jTtMn-joxPT0A,4,0,0,0,"5 stars for the location, it's absolutely wond...",2018-04-27 18:41:52
119,hz71QeBko3oBLqRD4lUygA,PR-NRdbE4Fnaq-kULI4gog,Agq4zoNLSIpT1_ZJbnrvww,4,0,0,0,The cost isn't expensive and it is very good. ...,2018-04-04 00:31:23
308,_ZNeVX7zMMlwNjc-W-vAeA,_5f0aOXWuPUJWg1FiCxt3Q,6yBizH8RnIYXk6vboLk3PA,3,0,0,0,With a suggestion from a friend on a place to ...,2018-06-19 07:43:25


In [18]:
review_json = portland_food_reviews_last_3_years.to_json(orient = "records")
review_parsed = json.loads(review_json)
review_parsed[0:3]

[{'review_id': 'drj-5euhU_5uVFcZ9nvsjw',
  'user_id': 'onoPTc_dcSlQOOk6PUxpaw',
  'business_id': 'g4CP3kgH1jTtMn-joxPT0A',
  'stars': 4,
  'useful': 0,
  'funny': 0,
  'cool': 0,
  'text': "5 stars for the location, it's absolutely wonderful. I ordered the shamrock bubble tea, which was one of the best I've ever had. I am subtracting a star for service however.  It was the barista that was so put out by waiting on us and then the 10-15 minutes it took to make the bubble tea... That's a bit long for a drink.",
  'date': 1524854512000},
 {'review_id': 'hz71QeBko3oBLqRD4lUygA',
  'user_id': 'PR-NRdbE4Fnaq-kULI4gog',
  'business_id': 'Agq4zoNLSIpT1_ZJbnrvww',
  'stars': 4,
  'useful': 0,
  'funny': 0,
  'cool': 0,
  'text': "The cost isn't expensive and it is very good. For me, places like Voodoo is too sweet for me, but it is not the case for Donut Palace. The donut itself is very airy and absolutely delicious when bitten down. I have only been there once, but I'm already enjoying it a lo

### save review data to json file

In [19]:
# #save food data to a new json file
# with open('data/yelp_academic_dataset_review_food.json', 'w') as f:
#     for obj in review_parsed:
#         json.dump(obj, f,  separators=(", ", ": "))
#         f.write('\n')